In [52]:
import loompy as lp
import seaborn as sns 

import os, glob, re, pickle
from functools import partial
from collections import OrderedDict
import operator as op
from cytoolz import compose

import pandas as pd
import seaborn as sns
import numpy as np
import scanpy as sc
import anndata as ad
import matplotlib as mpl
import matplotlib.pyplot as plt

from pyscenic.export import export2loom, add_scenic_metadata
from pyscenic.utils import load_motifs
from pyscenic.transform import df2regulons
from pyscenic.aucell import aucell
from pyscenic.binarization import binarize
from pyscenic.rss import regulon_specificity_scores
from pyscenic.plotting import plot_binarization, plot_rss

from IPython.display import HTML, display

In [2]:
#Resource
AUXILLIARIES_FOLDERNAME = "/home/jing/pySCENIC/resources/"
HUMAN_TFS_FNAME = os.path.join(AUXILLIARIES_FOLDERNAME, 'allTFs_mm.txt')

RANKING_DBS_FNAMES = list(map(lambda fn: os.path.join(AUXILLIARIES_FOLDERNAME, fn),
                       ['hg38_10kbp_up_10kbp_down_full_tx_v10_clust.genes_vs_motifs.rankings.feather',
                       'hg38_500bp_up_100bp_down_full_tx_v10_clust.genes_vs_motifs.rankings.feather']))

MOTIF_ANNOTATIONS_FNAME = os.path.join(AUXILLIARIES_FOLDERNAME, 'motifs-v10nr_clust-nr.hgnc-m0.001-o0.0.tbl')


In [ ]:
#Outputs
DATASET_ID = 'GBM'
RESULTS_FOLDERNAME = "/home/jing/Phd_project/project_GBM/gbm_OUTPUT/gbm_OUTPUT_publication/gbm_OUT_publication_scenic_run3/"
os.chdir(RESULTS_FOLDERNAME)
F_LOOM_SCE =  os.path.join(RESULTS_FOLDERNAME, '{}_filtered_scenic.loom'.format(DATASET_ID))
f_pyscenic_output = os.path.join(RESULTS_FOLDERNAME, '{}_final_scenic.loom'.format(DATASET_ID))


ADJACENCIES_FNAME = os.path.join(RESULTS_FOLDERNAME, '{}.adjacencies.tsv'.format(DATASET_ID))
MOTIFS_FNAME = os.path.join(RESULTS_FOLDERNAME, '{}.motifs.csv'.format(DATASET_ID))

In [ ]:
REGULONS_DAT_FNAME = os.path.join(RESULTS_FOLDERNAME, '{}.regulons.dat'.format(DATASET_ID))
AUCELL_MTX_FNAME = os.path.join(RESULTS_FOLDERNAME, '{}.auc.csv'.format(DATASET_ID))
BIN_MTX_FNAME = os.path.join(RESULTS_FOLDERNAME, '{}.bin.csv'.format(DATASET_ID))
THR_FNAME = os.path.join(RESULTS_FOLDERNAME, '{}.thresholds.csv'.format(DATASET_ID))

### Adata

In [3]:
directory = '/home/jing/Phd_project/project_GBM/gbm_DATA/gbm_DATA_GSE174554/gbm_DATA_scRNA_atlas'
os.chdir(directory)
names_list=['GSM5319518_SF2777','GSM5319548_SF2979','GSM5319519_SF2990',
                'GSM5319549_SF3073','GSM5319520_SF3076','GSM5319550_SF3243',
                'GSM5319521_SF3391','GSM5319551_SF3448','GSM5319511_SF11916',
                'GSM5319543_SF12382','GSM5319506_SF11082','GSM5319562_SF11488',
                'GSM5319530_SF9358','GSM5319568_SF9962','GSM5319559_SF9798','GSM5319532_SF9494']
adata_list = []

# Loop over each sample and read in the AnnData object
for name in names_list:

    mtx =f"{name}_matrix.mtx.gz"
    adata = sc.read_mtx(mtx)
    cells=pd.read_csv(f'{name}_barcodes.tsv.gz',header=None)
    features=pd.read_csv(f'{name}_features.tsv.gz',header=None,sep='\t')
    adata= adata.T
    #check the columns first to make sure they are the ones you need 
    adata.obs['CellID']= cells[0].tolist()
    adata.obs.index = adata.obs['CellID']
    adata.var['Gene']= features[0].tolist()
    adata.var.index= adata.var['Gene']
    adata.var_names_make_unique() 
    adata.var['mt'] =adata.var_names.str.startswith('MT-')


    sc.pp.filter_cells(adata, min_genes=200)
    sc.pp.calculate_qc_metrics(adata,qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
    adata= adata[adata.obs.n_genes_by_counts <6000, :]
    adata= adata[adata.obs.pct_counts_mt< 5, :].copy()


    adata.obs['source'] = name[11:]
    adata.layers["counts"] = adata.X.copy()    
    #sc.pp.normalize_total(adata, target_sum=1e4)
    #sc.pp.log1p(adata)
    #adata.raw = adata  # keep full dimension safe
    adata_list.append(adata)
batch_names = [adata.obs['source'].iloc[0] for adata in adata_list]
adata = adata_list[0].concatenate(adata_list[1:], batch_key='source', batch_categories=batch_names) 
adata.obs.index.name = "ref"
display(adata.obs)

/tmp/ipykernel_1053072/4194437479.py:40: FutureWarning: Use anndata.concat instead of AnnData.concatenate, AnnData.concatenate is deprecated and will be removed in the future. See the tutorial for concat at: https://anndata.readthedocs.io/en/latest/concatenation.html
  adata = adata_list[0].concatenate(adata_list[1:], batch_key='source', batch_categories=batch_names)


,CellID,n_genes,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,source
ref,,,,,,,
AAACCCAAGGGATCAC-1-SF2777,AAACCCAAGGGATCAC-1,468,468,579.0,9.0,1.554404,SF2777
AAACCCAGTCGATTTG-1-SF2777,AAACCCAGTCGATTTG-1,311,311,402.0,4.0,0.995025,SF2777
AAACCCAGTCGTCAGC-1-SF2777,AAACCCAGTCGTCAGC-1,864,864,1518.0,3.0,0.197628,SF2777
AAACCCAGTTGTAAAG-1-SF2777,AAACCCAGTTGTAAAG-1,632,632,891.0,12.0,1.346801,SF2777
AAACCCATCTATCGGA-1-SF2777,AAACCCATCTATCGGA-1,849,849,1338.0,43.0,3.213752,SF2777
...,...,...,...,...,...,...,...
TTTGGTTTCATTATCC-1-SF9494,TTTGGTTTCATTATCC-1,2027,2027,4502.0,0.0,0.000000,SF9494
TTTGGTTTCCCTCGTA-1-SF9494,TTTGGTTTCCCTCGTA-1,1972,1972,4071.0,0.0,0.000000,SF9494
TTTGTTGTCAGACAAA-1-SF9494,TTTGTTGTCAGACAAA-1,2340,2340,5272.0,0.0,0.000000,SF9494


In [4]:
outdir='/home/jing/Phd_project/project_GBM/gbm_OUTPUT/gbm_OUTPUT_publication'
doublet_df = pd.read_csv(os.path.join(outdir,'doublet_predictions.csv'), index_col=0)
doublet_df

for i in adata.obs.index:
    adata.obs.loc[i,'solo'] = doublet_df.loc[i,'Solo_Prediction']
adata_singlet = adata[adata.obs['solo']=='singlet']
adata_singlet

View of AnnData object with n_obs × n_vars = 50738 × 33694
    obs: 'CellID', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'source', 'solo'
    var: 'Gene', 'mt', 'n_cells_by_counts-SF11082', 'mean_counts-SF11082', 'pct_dropout_by_counts-SF11082', 'total_counts-SF11082', 'n_cells_by_counts-SF11488', 'mean_counts-SF11488', 'pct_dropout_by_counts-SF11488', 'total_counts-SF11488', 'n_cells_by_counts-SF11916', 'mean_counts-SF11916', 'pct_dropout_by_counts-SF11916', 'total_counts-SF11916', 'n_cells_by_counts-SF12382', 'mean_counts-SF12382', 'pct_dropout_by_counts-SF12382', 'total_counts-SF12382', 'n_cells_by_counts-SF2777', 'mean_counts-SF2777', 'pct_dropout_by_counts-SF2777', 'total_counts-SF2777', 'n_cells_by_counts-SF2979', 'mean_counts-SF2979', 'pct_dropout_by_counts-SF2979', 'total_counts-SF2979', 'n_cells_by_counts-SF2990', 'mean_counts-SF2990', 'pct_dropout_by_counts-SF2990', 'total_counts-SF2990', 'n_cells_by_counts-SF3073', 'mean_counts-SF3073

In [12]:
# create basic row and column attributes for the loom file:
row_attrs = {
    "Gene": np.array(adata_singlet.var_names) ,
}
col_attrs = {
    "CellID": np.array(adata_singlet.obs_names) ,
    "nGene": np.array( np.sum(adata_singlet.X.transpose()>0 , axis=0)).flatten() ,
    "nUMI": np.array( np.sum(adata_singlet.X.transpose() , axis=0)).flatten() ,
}
lp.create( F_LOOM_SCE, adata_singlet.X.transpose(), row_attrs, col_attrs)

### SECNIC inference

In [19]:
#I) GRN
!pyscenic grn {F_LOOM_SCE} {HUMAN_TFS_FNAME} -o {ADJACENCIES_FNAME} --num_workers 20


2025-05-21 11:52:05,911 - pyscenic.cli.pyscenic - INFO - Loading expression matrix.

2025-05-21 11:52:16,687 - pyscenic.cli.pyscenic - INFO - Inferring regulatory networks.
preparing dask client
parsing input
creating dask graph
20 partitions
computing dask graph
/home/jing/miniforge3/envs/pyscenic/lib/python3.10/site-packages/distributed/client.py:3169: UserWarning: Sending large graph of size 5.64 GiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
not shutting down client, client was created externally
finished

2025-05-21 14:51:32,923 - pyscenic.cli.pyscenic - INFO - Writing results to file.


In [20]:
DBS_PARAM = ' '.join(RANKING_DBS_FNAMES)

In [22]:
#II) CTX
!pyscenic ctx {ADJACENCIES_FNAME} \
    {DBS_PARAM} \
    --annotations_fname {MOTIF_ANNOTATIONS_FNAME} \
    --expression_mtx_fname {F_LOOM_SCE} \
    --output reg.csv \
    --mask_dropouts \
    --num_workers 20


2025-05-21 16:42:35,211 - pyscenic.cli.pyscenic - INFO - Creating modules.

2025-05-21 16:42:35,867 - pyscenic.cli.pyscenic - INFO - Loading expression matrix.

2025-05-21 16:43:02,871 - pyscenic.utils - INFO - Calculating Pearson correlations.

2025-05-21 16:43:03,076 - pyscenic.utils - WARNING - Note on correlation calculation: the default behaviour for calculating the correlations has changed after pySCENIC verion 0.9.16. Previously, the default was to calculate the correlation between a TF and target gene using only cells with non-zero expression values (mask_dropouts=True). The current default is now to use all cells to match the behavior of the R verision of SCENIC. The original settings can be retained by setting 'rho_mask_dropouts=True' in the modules_from_adjacencies function, or '--mask_dropouts' from the CLI.
	Dropout masking is currently set to [True].

2025-05-21 16:49:14,651 - pyscenic.utils - INFO - Creating modules.

2025-05-21 16:49:52,818 - pyscenic.cli.pyscenic - IN

In [ ]:
#III) AUCELL
!pyscenic aucell \
    {F_LOOM_SCE} \
    reg.csv \
    --output {f_pyscenic_output} \
    --num_workers 20


In [ ]:
import json
import zlib
import base64

# collect SCENIC AUCell output
lf = lp.connect( f_pyscenic_output, mode='r+', validate=False )
auc_mtx = pd.DataFrame( lf.ca.RegulonsAUC, index=lf.ca.CellID)
lf.close()
AUCELL_MTX_FNAME = os.path.join(outdir, 'auc.csv')
auc_mtx.to_csv(AUCELL_MTX_FNAME)


In [54]:
def derive_regulons(motifs, db_names=('hg38_10kbp_up_10kbp_down', 
                                 'hg38_500bp_up_100bp')):
    motifs.columns = motifs.columns.droplevel(0)

    def contains(*elems):
        def f(context):
            return any(elem in context for elem in elems)
        return f

    # For the creation of regulons we only keep the 10-species databases and the activating modules. We also remove the
    # enriched motifs for the modules that were created using the method 'weight>50.0%' (because these modules are not part
    # of the default settings of modules_from_adjacencies anymore.
    motifs = motifs[
        np.fromiter(map(compose(op.not_, contains('weight>50.0%')), motifs.Context), dtype=np.bool) & \
        np.fromiter(map(contains(*db_names), motifs.Context), dtype=np.bool) & \
        np.fromiter(map(contains('activating'), motifs.Context), dtype=np.bool)]

    # We build regulons only using enriched motifs with a NES of 3.0 or higher; we take only directly annotated TFs or TF annotated
    # for an orthologous gene into account; and we only keep regulons with at least 10 genes.
    regulons = list(filter(lambda r: len(r) >= 10, df2regulons(motifs[(motifs['NES'] >= 3.0) 
                                                                      & ((motifs['Annotation'] == 'gene is directly annotated')
                                                                        | (motifs['Annotation'].str.startswith('gene is orthologous to')
                                                                           & motifs['Annotation'].str.endswith('which is directly annotated for motif')))
                                                                     ])))
    
    # Rename regulons, i.e. remove suffix.
    return list(map(lambda r: r.rename(r.transcription_factor), regulons))

In [ ]:
df_motifs = load_motifs(MOTIFS_FNAME)
regulons = derive_regulons(df_motifs)

In [30]:
from pyscenic.utils import load_motifs
df_motifs = load_motifs(MOTIFS_FNAME)
df_motifs

In [ ]:
modules = list(modules_from_adjacencies(ADJACENCIES_FNAME, F_LOOM_SCE))

In [ ]:
# Calculate a list of enriched motifs and the corresponding target genes for all modules.
with ProgressBar():
    df = prune2df(dbs, modules, MOTIF_ANNOTATIONS_FNAME)

# Create regulons from this table of enriched motifs.
regulons = df2regulons(df)

In [44]:
REGULONS_FNAME = os.path.join(RESULTS_FOLDERNAME, "regulons.p")

In [45]:
import pickle
df = load_motifs(MOTIFS_FNAME)
with open(REGULONS_FNAME, "rb") as f:
    regulons = pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: '/home/jing/Phd_project/project_GBM/gbm_OUTPUT/gbm_OUTPUT_publication/gbm_OUT_publication_scenic_run3/regulons.p'

In [37]:
BASE_URL = "http://motifcollections.aertslab.org/v9/logos/"
COLUMN_NAME_LOGO = "MotifLogo"
COLUMN_NAME_MOTIF_ID = "MotifID"
COLUMN_NAME_TARGETS = "TargetGenes"

In [38]:

def display_logos(df: pd.DataFrame, top_target_genes: int = 3, base_url: str = BASE_URL):
    """
    :param df:
    :param base_url:
    """
    # Make sure the original dataframe is not altered.
    df = df.copy()
    
    # Add column with URLs to sequence logo.
    def create_url(motif_id):
        return '<img src="{}{}.png" style="max-height:124px;"></img>'.format(base_url, motif_id)
    df[("Enrichment", COLUMN_NAME_LOGO)] = list(map(create_url, df.index.get_level_values(COLUMN_NAME_MOTIF_ID)))
    
    # Truncate TargetGenes.
    def truncate(col_val):
        return sorted(col_val, key=op.itemgetter(1))[:top_target_genes]
    df[("Enrichment", COLUMN_NAME_TARGETS)] = list(map(truncate, df[("Enrichment", COLUMN_NAME_TARGETS)]))
    
    MAX_COL_WIDTH = pd.get_option('display.max_colwidth')
    pd.set_option('display.max_colwidth', -1)
    display(HTML(df.head().to_html(escape=False)))
    pd.set_option('display.max_colwidth', MAX_COL_WIDTH)



In [27]:
!pyscenic aucell \
    {F_LOOM_SCE} \
    {MOTIFS_FNAME} \
    --output {f_pyscenic_output} \
    --num_workers 20


2025-05-21 17:13:32,364 - pyscenic.cli.pyscenic - INFO - Loading expression matrix.

2025-05-21 17:13:43,347 - pyscenic.cli.pyscenic - INFO - Loading gene signatures.
Create regulons from a dataframe of enriched features.
Additional columns saved: []

2025-05-21 17:13:43,494 - pyscenic.cli.pyscenic - INFO - Calculating cellular enrichment.


In [25]:
auc_mtx = pd.read_csv(AUCELL_MTX_FNAME, index_col=0)

FileNotFoundError: [Errno 2] No such file or directory: '/home/jing/Phd_project/project_GBM/gbm_OUTPUT/gbm_OUTPUT_publication/gbm_OUT_publication_scenic_run3/GBM.auc.csv'

In [ ]:
from pyscenic.export import export2loom, add_scenic_metadata
from pyscenic.utils import load_motifs
from pyscenic.transform import df2regulons
from pyscenic.aucell import aucell
from pyscenic.binarization import binarize
from pyscenic.rss import regulon_specificity_scores
from pyscenic.plotting import plot_binarization, plot_rss


In [ ]:
bin_mtx, thresholds = binarize(auc_mtx) bin_mtx.to_csv(BIN_MTX_FNAME) thresholds.to_frame().rename(columns={0:'threshold'}).to_csv(THR_FNAME) 

In [ ]:
bin_mtx = pd.read_csv(BIN_MTX_FNAME, index_col=0)
thresholds = pd.read_csv(THR_FNAME, index_col=0).threshold

In [ ]:
plot_binarization(auc_mtx, 'NFKB2', thresholds['NFKB2'], ax=ax1)
plot_binarization(auc_mtx, 'MITF', thresholds['MITF'], ax=ax2)
plot_binarization(auc_mtx, 'FOXP3', thresholds['FOXP3'], ax=ax3)
plot_binarization(auc_mtx, 'PAX5', thresholds['PAX5'], ax=ax4)
plot_binarization(auc_mtx, 'IRF8', thresholds['IRF8'], ax=ax5)
plot_binarization(auc_mtx, 'IRF3', thresholds['IRF3'], ax=ax6)
plot_binarization(auc_mtx, 'MLX', thresholds['MLX'], ax=ax7)
plot_binarization(auc_mtx, 'YY1', thresholds['YY1'], ax=ax8)